In [ ]:
# the demodulation part of this code was submitted as an assignment for CS4020 course in 2023 in University of South Eastern Norway (USN)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
from scipy.signal import butter, sosfreqz, sosfilt, hilbert
from scipy.fft import fft, fftfreq
import re

#  band pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def bandpass_signal(signal, fs, lowcut, highcut):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(2, [low, high], btype='band', analog=False, output='sos')
    y = sosfilt(sos, signal)
    return y

#  low pass filter (part of demodualtion)
#This part of code is adapted from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def low_signal(signal, lowcut, fs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    order = 2
    sos1 = butter(order, low, btype='low', output='sos')
    l = sosfilt(sos1, signal)
    return l

# Calculate the polar spectrum and return envelope and amplitude using FFT
def calculate_polar_spectrum(data, fs, lowcut, highcut, expected_freq):

    #demodulation
    filtered_signal = bandpass_signal(data, fs, lowcut, highcut)   
    analytic_signal = hilbert(filtered_signal)
    envelope = np.abs(analytic_signal)
    lowpass_signal = low_signal(envelope, lowcut, fs)
    n = len(lowpass_signal)
    fft_result = fft(lowpass_signal)
    freqs = fftfreq(n, 1/fs)[:n//2]
    amplitude = 2.0 / n * np.abs(fft_result[0:n//2])

    return envelope, freqs, amplitude

def newdataset(path, date, location, first, last):
    fs = 20000
    lowcut = 2000
    highcut = 9900
    expected_freq =236.4

    file_path = os.path.join(path, date)

    # Load the vibration data
    dataset = pd.read_csv(file_path, sep='\t', header=None)
    data = dataset.iloc[:, 0].values

    # get envelope, freqs, and amplitude
    envelope, freqs, amplitude = calculate_polar_spectrum(data, fs, lowcut, highcut, expected_freq)

    #This part of code is adapted and modified from the following github repository : https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, polar=True)
    colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white', 'purple', 'orange']

   # making larger circles
    num_points_outer = 100  
    num_points_inner = 500  

    for i in range(len(amplitude)):
        if freqs[i] <= expected_freq:  
            if i < len(amplitude) // 2:
                r_values = np.full(num_points_outer, amplitude[i])
                num_points = num_points_outer
            else:
                r_values = np.full(num_points_inner, amplitude[i] * 10)  
                num_points = num_points_inner

            theta_values = np.linspace(0, 2 * np.pi, num_points) + (i * np.pi / len(amplitude))  

            ax.plot(theta_values, r_values, color=colors[i % len(colors)], linewidth=1.5)  
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_rticks([])
    ax.spines['polar'].set_visible(False)
    ax.set_title("Polar Spectrum")

    plt.tight_layout()  

    # Saving polars
    directory_path = r"C:\Users\zahra\nasa\filter\polar13" 
    file_name = f"{date}_polar_spectrum.png"
    file_path = os.path.join(directory_path, file_name)
    plt.savefig(file_path, dpi=300)
    plt.close() 

# Example usage
path = "C:/Users/zahra/Thesis/datasets/2nd_test"
dates = os.listdir(path)
for date in dates[-465:] : #based on initial detected fault
    location = dates.index(date)
    newdataset(path, date, location, dates[0], dates[-1])
